# Imports

In [1]:
import numpy
numpy.random.seed(123)
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn import neighbors
from sklearn.preprocessing import Normalizer

import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from tqdm.auto import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu'

c:\Users\Sam\Desktop\ML Proj\MLProj\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Function to fetch previously saved embeddings

In [2]:
def embed_features(X, saved_embeddings_fname):
    # f_embeddings = open("embeddings_shuffled.pickle", "rb")
    f_embeddings = open(saved_embeddings_fname, "rb") # Open the pickle file
    embeddings = pickle.load(f_embeddings) # Load it

    index_embedding_mapping = {1: 0, 2: 1, 4: 2, 5: 3, 6: 4, 7: 5} # The values are the indices of the embedded features
    X_embedded = []

    (num_records, num_features) = X.shape
    for record in X:
        embedded_features = []
        for i, feat in enumerate(record):
            feat = int(feat)
            if i not in index_embedding_mapping.keys():
                embedded_features += [feat]
            else:
                embedding_index = index_embedding_mapping[i]
                embedded_features += embeddings[embedding_index][feat].tolist()

        X_embedded.append(embedded_features)

    return numpy.array(X_embedded)

### This function takes the input data and returns a list of each column 

In [3]:
def split_features(X):
    X_list = []

    store_index = X[..., [1]]
    X_list.append(store_index)

    day_of_week = X[..., [2]]
    X_list.append(day_of_week)

    promo = X[..., [3]]
    X_list.append(promo)

    year = X[..., [4]]
    X_list.append(year)

    month = X[..., [5]]
    X_list.append(month)

    day = X[..., [6]]
    X_list.append(day)

    State = X[..., [7]]
    X_list.append(State)

    return numpy.array(X_list)

# Base Model Class

In [4]:
class Model(nn.Module):
    def evaluate(self, X_val, y_val):
        assert(min(y_val) > 0) # All sales are positive so predictions should be positive
        guessed_sales = self.guess(X_val)
        relative_err = numpy.absolute((y_val - guessed_sales) / y_val)
        result = numpy.sum(relative_err) / len(y_val)
        return result

In [5]:
class LinearModel(Model):
    def __init__(self, X_train, y_train, X_val, y_val):
        self.clf = linear_model.LinearRegression() # this is from sklearn not keras or tensorflow so please don't fail us
        self.clf.fit(X_train, numpy.log(y_train))
        print("Result on validation data: ", self.evaluate(X_val, y_val))

    def guess(self, feature):
        return numpy.exp(self.clf.predict(feature))

In [6]:
class RF(Model):
    def __init__(self, X_train, y_train, X_val, y_val):
        self.clf = RandomForestRegressor(n_estimators=200, verbose=True, max_depth=35, min_samples_split=2,
                                         min_samples_leaf=1) # This is from sklearn not keras or tensorflow so please don't fail us
        self.clf.fit(X_train, numpy.log(y_train))
        print("Result on validation data: ", self.evaluate(X_val, y_val))

    def guess(self, feature):
        return numpy.exp(self.clf.predict(feature))

In [7]:
# Implement support vector machine regression because they used sklearn, i copied it please dont DC us
class SVM(Model):
    def __init__(self, X_train, y_train, X_val, y_val):
        self.X_train = X_train
        self.y_train = y_train
        self.__normalize_data()
        self.clf = SVR(kernel='linear', degree=3, gamma='auto', coef0=0.0, tol=0.001,
                    C=1.0, epsilon=0.1, shrinking=True, cache_size=200, verbose=False, max_iter=-1)

        self.clf.fit(self.X_train, torch.log(self.y_train))
        print("Result on validation data: ", self.evaluate(X_val, y_val))

    def __normalize_data(self):
        self.scaler = StandardScaler()
        self.X_train = self.scaler.fit_transform(self.X_train)

    def guess(self, feature):
        return torch.exp(self.clf.predict(feature))

In [8]:
class XGBoost(Model):

    def __init__(self, X_train, y_train, X_val, y_val):
        dtrain = xgb.DMatrix(X_train, label= torch.log(y_train))
        evallist = [(dtrain, 'train')]
        param = {'nthread': -1,
                'max_depth': 7,
                'eta': 0.02,
                'silent': 1,
                'objective': 'reg:linear',
                'colsample_bytree': 0.7,
                'subsample': 0.7}
        num_round = 3000
        self.bst = xgb.train(param, dtrain, num_round, evallist)
        print("Result on validation data: ", self.evaluate(X_val, y_val))

    def guess(self, feature):
        dtest = xgb.DMatrix(feature)
        return numpy.exp(self.bst.predict(dtest))

In [9]:
class HistricalMedian(Model):
    def __init__(self, X_train, y_train, X_val, y_val):
        self.history = {}
        self.feature_index = [1, 2, 3, 4]
        for x, y in zip(X_train, y_train):
            key = tuple(x[self.feature_index])
            self.history.setdefault(key, []).append(y)
        print("Result on validation data: ", self.evaluate(X_val, y_val))

    def guess(self, features):
        features = numpy.array(features)
        features = features[:, self.feature_index]
        guessed_sales = [numpy.median(self.history[tuple(feature)]) for feature in features]
        return numpy.array(guessed_sales)

In [10]:
class KNN(Model):
    def __init__(self, X_train, y_train, X_val, y_val):
        self.normalizer = Normalizer()
        self.normalizer.fit(X_train)
        self.clf = neighbors.KNeighborsRegressor(n_neighbors=10, weights='distance', p=1)
        self.clf.fit(self.normalizer.transform(X_train), numpy.log(y_train))
        print("Result on validation data: ", self.evaluate(self.normalizer.transform(X_val), y_val))

    def guess(self, feature):
        return numpy.exp(self.clf.predict(self.normalizer.transform(feature)))

In [11]:
class NN_with_EntityEmbedding(Model):
    def __init__(self, X_train, y_train, X_val, y_val):
        super().__init__()
        self.epochs = 10
        self.max_log_y = max(torch.max(torch.log(y_train)), torch.max(torch.log(y_val)))
        self.one = nn.Sequential(nn.Embedding(1115, 10), nn.Flatten())
        self.two = nn.Sequential(nn.Embedding(7, 6), nn.Flatten())
        self.three = nn.Linear(1,1)
        self.four = nn.Sequential(nn.Embedding(3, 2), nn.Flatten())
        self.five = nn.Sequential(nn.Embedding(12, 6), nn.Flatten())
        self.six = nn.Sequential(nn.Embedding(31, 10), nn.Flatten())
        self.seven = nn.Sequential(nn.Embedding(12, 6), nn.Flatten())
        self.network = nn.Sequential(
            nn.Linear(41, 1000),
            nn.ReLU(),
            nn.Linear(1000, 500),
            nn.ReLU(),
            nn.Linear(500, 1),
            nn.Sigmoid()
        )
        nn.init.uniform_(self.network[0].weight, -0.05, 0.05)
        nn.init.uniform_(self.network[2].weight, -0.05, 0.05)
        nn.init.xavier_uniform_(self.network[4].weight)
        nn.init.xavier_uniform_(self.three.weight)
        nn.init.uniform_(self.one[0].weight, -0.05, 0.05)
        nn.init.uniform_(self.two[0].weight, -0.05, 0.05)
        nn.init.uniform_(self.four[0].weight, -0.05, 0.05)
        nn.init.uniform_(self.five[0].weight, -0.05, 0.05)
        nn.init.uniform_(self.six[0].weight, -0.05, 0.05)
        nn.init.uniform_(self.seven[0].weight, -0.05, 0.05)
        self.to(device)
        self.fit(X_train,y_train, X_val, y_val)

    def forward(self, x):
        one = self.one(x[:,[1]])
        two = self.two(x[:,[2]])
        three=self.three(x[:, [3]].float())
        four=self.four(x[:, [4]])
        five=self.five(x[:,[5]])
        six=self.six(x[:, [6]])
        seven = self.seven(x[:, [7]])
        concat = torch.cat([one, two, three, four, five, six, seven], dim=1)
        output = self.network(concat)
        return output
    
    def _val_for_fit(self, val):
        return torch.log(val)/self.max_log_y
    
    def _val_for_pred(self, val):
        return torch.exp(val*self.max_log_y)
    
    def fit(self, X_train, y_train, X_val, y_val):
        loss_fn = nn.L1Loss().to(device)
        optimizer = optim.Adam(self.parameters(), lr=0.001, eps=1e-07)
        train_data = TensorDataset(X_train, self._val_for_fit(y_train))
        train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
        for epoch in range(self.epochs):
            for inputs, targets in tqdm(train_loader,desc=f"Training epoch {epoch+1}/{self.epochs}",leave=False):
                inputs = inputs.to(device)
                targets = targets.to(device)
                outputs = self.forward(inputs).squeeze()
                loss = loss_fn(outputs, targets)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        with torch.inference_mode():
            print(f"Loss on validation data: ", self.evaluate(X_val.to(device), y_val.to(device)))

    def evaluate(self, X_val, y_val):
        with torch.inference_mode():
            assert(min(y_val) > 0) # All sales are positive so predictions should be positive
            guessed_sales = self.guess(X_val) # Guess is implemented in children classes for inference
            relative_err = torch.absolute((y_val - guessed_sales) / y_val) 
            result = torch.sum(relative_err) / len(y_val)
            return result.item()
    
    def guess(self, features):
        with torch.inference_mode():
            result = self.forward(features).flatten()
        return self._val_for_pred(result)

In [12]:
with open('feature_train_data.pickle', 'rb') as f:
    X, y = pickle.load(f)

numpy.random.seed(123)
torch.manual_seed(123)

X = torch.tensor(X)
y = torch.tensor(y)
train_ratio = 0.9
num_records = len(X)
train_size = int(train_ratio * num_records)

shuffle_data = False

if shuffle_data:
    print("Using shuffled data")
    sh = numpy.arange(X.shape[0])
    numpy.random.shuffle(sh)
    X = X[sh]
    y = y[sh]

X_train = X[:train_size]
X_val = X[train_size:]
y_train = y[:train_size]
y_val = y[train_size:]

def sample(X, y, n):
    '''random samples'''
    num_row = X.shape[0]
    indices = numpy.random.randint(num_row, size=n)
    return X[indices, :], y[indices]

X_train, y_train = sample(X_train, y_train, 200000)
# from sklearn.model_selection import train_test_split
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

models = []

print("Fitting NN_with_EntityEmbedding...")
for i in range(5):
    models.append(NN_with_EntityEmbedding(X_train, y_train, X_val, y_val))

Fitting NN_with_EntityEmbedding...


Training epoch 1/10:   0%|          | 0/1563 [00:00<?, ?it/s]

Loss on validation data:  0.12816597521305084


Loss on validation data:  0.10211324691772461


Loss on validation data:  0.11393663287162781


Loss on validation data:  0.10134878009557724


Loss on validation data:  0.11169911175966263


In [13]:
def evaluate_models(models, X, y):
    assert(min(y) > 0)
    guessed_sales = torch.stack([model.guess(X) for model in models])
    mean_sales = guessed_sales.mean(axis=0)
    relative_err = torch.absolute((y - mean_sales) / y)
    result = torch.sum(relative_err) / len(y)
    return result.item()

print("Evaluate combined models...")
print("Training error...")
r_train = evaluate_models(models, X_train.to(device), y_train.to(device))
print(r_train)

print("Validation error...")
r_val = evaluate_models(models, X_val.to(device), y_val.to(device))
print(r_val)

Evaluate combined models...
Training error...
tensor(0.0657, device='cuda:0')
Validation error...
tensor(0.1000, device='cuda:0')


In [14]:
class NN(Model):
    def __init__(self, X_train, y_train, X_val, y_val):
        super(NN, self).__init__()
        self.epochs = 10
        self.max_log_y = max(torch.max(torch.log(y_train)), torch.max(torch.log(y_val)))
        self.model = nn.Sequential(
            nn.Linear(1183, 1000),
            nn.ReLU(),
            nn.Linear(1000, 500),
            nn.ReLU(),
            nn.Linear(500, 1),
            nn.Sigmoid(),
        )
        self.to(device)
        self.fit(X_train,y_train, X_val, y_val)
        

    def forward(self, x):
        return self.model(x)

    def _val_for_fit(self, val):
        return torch.log(val) / self.max_log_y

    def _val_for_pred(self, val):
        return torch.exp(val * self.max_log_y)

    def fit(self, X_train, y_train, X_val, y_val):
        loss_fn = nn.L1Loss().to(device)
        optimizer = optim.Adam(self.parameters(), lr=0.001)
        train_data = TensorDataset(X_train, self._val_for_fit(y_train))
        train_loader = DataLoader(train_data, batch_size=128, shuffle=False)
        for epoch in range(self.epochs):
            for inputs, targets in train_loader:
                inputs = inputs.float().to(device)
                targets = targets.to(device)
                outputs = self.forward(inputs).squeeze()
                loss = loss_fn(outputs, targets)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            with torch.inference_mode():
                print(
                    "Result on validation data: ",
                    self.evaluate(X_val.float().to(device), y_val.to(device)),
                )

    def evaluate(self, X_val, y_val):
        assert (
            min(y_val) > 0
        )  # All sales are positive so predictions should be positive
        guessed_sales = self.guess(
            X_val
        )  # Guess is implemented in children classes for inference
        relative_err = torch.absolute((y_val - guessed_sales) / y_val)
        result = torch.sum(relative_err) / len(y_val)
        return result

    def guess(self, features):
        with torch.no_grad():
            result = self.model(features).flatten()
        return self._val_for_pred(result)

In [15]:
model = NN(X_train, y_train, X_val, y_val)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (128x8 and 1183x1000)